In [47]:
import pandas_datareader.data as web
import datetime
import pandas as pd
from functools import reduce


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

portfolio = pd.read_csv("data.csv")
# take only first 10 columns
portfolio = portfolio.iloc[:, 1:10]
print(portfolio.head())

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientCVaR

mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()

S = portfolio.cov()
ef_cvar = EfficientCVaR(mu, S)
cvar_weights = ef_cvar.min_cvar()
cleaned_weights = ef_cvar.clean_weights()
print(dict(cleaned_weights))

print(ef_cvar.portfolio_performance(verbose=True))



       A    AAPL     ABC   ABMD    ABT   ADBE    ADM   ADSK    AEE
0  41.81  105.26  103.71  90.28  44.91  93.94  36.68  60.93  43.23
1  40.69  105.35  101.87  85.24  42.93  91.97  35.84  60.31  43.03
2  40.55  102.71  103.36  85.00  42.92  92.34  36.24  60.39  43.54
3  40.73  100.70  101.78  85.30  42.56  91.02  35.36  59.54  43.82
4  39.00   96.45   98.53  81.92  41.54  89.11  34.52  55.98  43.85
{'A': 0.0, 'AAPL': 0.0, 'ABC': 0.0, 'ABMD': 1.0, 'ABT': 0.0, 'ADBE': 0.0, 'ADM': 0.0, 'ADSK': 0.0, 'AEE': 0.0}
Expected annual return: 51.3%
Conditional Value at Risk: -16929.92%
(0.5134385268827328, -169.29917946437817)


/home/codeubuntu/intel/oneapi/intelpython/latest/lib/python3.9/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
